In [1]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  

In [2]:
import matplotlib.pyplot as plt
#import statsmodels.tsa.seasonal as smt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
import datetime as dt
from sklearn import linear_model 

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
#print(check_output(["ls", "../input"]).decode("utf8"))
import os
data_path = "/notebooks/stock_data_analysis/data_2/Stocks"
os.chdir(data_path)


In [3]:
data_path

'/notebooks/stock_data_analysis/data_2/Stocks'

In [4]:
#read data
# kernels let us navigate through the zipfile as if it were a directory

# trying to read a file of size zero will throw an error, so skip them
filenames = [x for x in os.listdir(data_path) if x.endswith('.txt') and os.path.getsize(x) > 0]
# = random.sample(filenames,1)
#print(filenames)

In [5]:
from sklearn.preprocessing import MinMaxScaler
def data_normalize(df, col):
    data_close = np.reshape(df[col], (df[col].shape[0], 1))
    scaler = MinMaxScaler()
    result_get = scaler.fit_transform(data_close)
    return result_get

def binning_data(data, num_embedding = 1000):
    bins = np.linspace(0, 1, num_embedding)
    inds = np.digitize(data_get, bins)
    return inds


def get_data(file_name):
    file_name = os.path.join('/notebooks/stock_data_analysis/data_2/Stocks', file_name)
    df = pd.read_csv(file_name, sep=',')
    #combine attributes
    #combine high and low by avg
    #combine open and close by avg
    #combine avgHighLow and avgOpenClose
    df['Price'] = (df['High'] + df['Low'] + df['Open'] + df['Close'])/4
    #take log as this flattens the data more, resulting in a better prediction
    df['Price'] = np.log(df['Price'])

    #drop obsolete columns for faster processing
    columns2Drop = [] #['High', 'Low', 'Open', 'Close', 'OpenInt']
    df = df.drop(labels=columns2Drop, axis=1)

    #create new attribute of "movement"
    df['Volume*Price'] = df['Volume'] * df['Price']
    # print(df)

    label = filename
    df['Label'] = label
    df['Date'] = pd.to_datetime(df['Date'])
    #conver data to an number so we can fit it to LinearRegression()
#     df['Date'] = df['Date'].map(dt.datetime.toordinal)

    #data.append(df)
    return df

In [6]:
def load_pickle(data_path):
    with open(data_path, 'rb') as handle:
        data_get = pickle.load(handle)
    return data_get

def save_picle(data_path, data):
    with open(data_path, 'wb') as handle:
        pickle.dump(data, handle)



In [7]:
def exists(path):
    """Test whether a path exists.  Returns False for broken symbolic links"""
    try:
        st = os.stat(path)
    except os.error:
        return False
    return True

In [8]:
pathModel = os.getcwd()  #
embedding_col_path = os.path.join(pathModel, "data_get.txt")

In [9]:
flag = exists(embedding_col_path)

In [10]:
import json

In [34]:
if not flag:
    data = []
    for filename in filenames:
        df = get_data(filename)
        binning_sentence = data_normalize(df, 'Close')
        inds = list(binning_data(binning_sentence).flat)
        data.append(inds)
    with open(embedding_col_path, 'w') as f:
        f.write(json.dumps(data))
else:
    #Now read the file back into a Python list object
    with open(embedding_col_path, 'r') as f:
        data = json.loads(f.read())  
    
    
    

In [24]:
import tensorflow as tf

batch_size = 256
embedding_size = 64
generations = 50000
print_loss_every = 500
num_embedding = 1000
num_sampled = int(batch_size/2)    # Number of negative examples to sample.
window_size = 8       # How many words to consider left and right.

In [25]:
valid_examples = [1,20,30,149,293]


In [26]:
# Generate data randomly (N words behind, target, N words ahead)
def generate_batch_data(sentences, batch_size, window_size, method='skip_gram'):
    # Fill up data batch
    batch_data = []
    label_data = []
    while len(batch_data) < batch_size:
        # select random sentence to start
        idx = np.random.choice(len(sentences),1)
        rand_sentence = sentences[idx[0]]
        # Generate consecutive windows to look at
        window_sequences = [rand_sentence[max((ix-window_size),0):(ix+window_size+1)] for ix, x in enumerate(rand_sentence)]
        # Denote which element of each window is the center word of interest
        label_indices = [ix if ix<window_size else window_size for ix,x in enumerate(window_sequences)]
        
        # Pull out center word of interest for each window and create a tuple for each window
        if method=='skip_gram':
            batch_and_labels = [(x[y], x[:y] + x[(y+1):]) for x,y in zip(window_sequences, label_indices)]
            # Make it in to a big list of tuples (target word, surrounding word)
            tuple_data = [(x, y_) for x,y in batch_and_labels for y_ in y]
        elif method=='cbow':
            batch_and_labels = [(x[:y] + x[(y+1):], x[y]) for x,y in zip(window_sequences, label_indices)]
            # Make it in to a big list of tuples (target word, surrounding word)
            tuple_data = [(x_, y) for x,y in batch_and_labels for x_ in x]
        else:
            raise ValueError('Method {} not implemented yet.'.format(method))
            
        # extract batch and labels
        batch, labels = [list(x) for x in zip(*tuple_data)]
        batch_data.extend(batch[:batch_size])
        label_data.extend(labels[:batch_size])
    # Trim batch and label at the end
    batch_data = batch_data[:batch_size]
    label_data = label_data[:batch_size]
    
    # Convert to numpy array
    batch_data = np.array(batch_data)
    label_data = np.transpose(np.array([label_data]).astype(int))
    
    return(batch_data, label_data)

In [27]:
# Define Embeddings:
embeddings = tf.Variable(tf.random_uniform([num_embedding, embedding_size], -1.0, 1.0), name = "product_vec")

# NCE loss parameters
nce_weights = tf.Variable(tf.truncated_normal([num_embedding, embedding_size],
                                               stddev=1.0 / np.sqrt(embedding_size)))
nce_biases = tf.Variable(tf.zeros([num_embedding]))

# Create data/target placeholders
x_inputs = tf.placeholder(tf.int32, shape=[batch_size])
y_target = tf.placeholder(tf.int32, shape=[batch_size, 1])
valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

# Lookup the word embedding:
embed = tf.nn.embedding_lookup(embeddings, x_inputs)

# Get loss from prediction
loss_get = tf.nn.nce_loss(weights=nce_weights,            # Tensor of shape(50000, 128)
                          biases=nce_biases,              # vector of zeros; len(128)
                          labels=y_target,            # labels == context words enums
                          inputs=embed,                   # Tensor of shape(128, 128)
                          num_sampled=num_sampled,        # 64: randomly chosen negative (rare) words
                          num_classes=num_embedding)   # 50000: by construction
loss = tf.reduce_mean(loss_get)



In [28]:
# Create optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1.0).minimize(loss)

tf.summary.scalar("loss", loss)

merged_summary_op = tf.summary.merge_all()

In [29]:
# Cosine similarity between words
norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
normalized_embeddings = embeddings / norm
valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)




In [30]:
config = tf.ConfigProto()


try:
    sess = tf.Session(config=config)
except:
    sess = tf.Session(config=config)

In [31]:
#Add variable initializer.
init = tf.global_variables_initializer()
sess.run(init)

In [35]:
LOG_DIR = '/tmp/testing/stock2vec_1'
#tensorboard --logdir=/tmp/testing/example_2_2
summary_writer = tf.summary.FileWriter(LOG_DIR, graph=tf.get_default_graph())
print_valid_every = 100
# Run the skip gram model.
valid_words = [1,23,10,300]
loss_vec = []
loss_x_vec = []
for i in range(generations):
    batch_inputs, batch_labels = generate_batch_data(data, batch_size, window_size)
    feed_dict = {x_inputs : batch_inputs, y_target : batch_labels}

    # Run the train step
    _, summary = sess.run([optimizer, merged_summary_op], feed_dict=feed_dict)
    
    summary_writer.add_summary(summary, i)

    # Return the loss
    if (i+1) % print_loss_every == 0:
        loss_val = sess.run(loss, feed_dict=feed_dict)
        loss_vec.append(loss_val)
        loss_x_vec.append(i+1)
        print("Loss at step {} : {}".format(i+1, loss_val))
      
    # Validation: Print some random words and top 5 related words
#     if (i+1) % print_valid_every == 0:
#         sim = sess.run(similarity, feed_dict=feed_dict)
#         for j in range(len(valid_words)):
#             valid_word = name_dict[word_dictionary_rev[valid_examples[0]]] #word_dictionary_rev[valid_examples[j]]
#             top_k = 5 # number of nearest neighbors
#             nearest = (-sim[j, :]).argsort()[1:top_k+1]
#             #name_dict[word_dictionary_rev[valid_examples[0]]]
#             log_str = "Nearest to {}::".format(valid_word)
#             for k in range(top_k):
#                 close_word = name_dict[word_dictionary_rev[valid_examples[k]]] #word_dictionary_rev[nearest[k]]
#                 log_str = "%s %s," % (log_str, close_word)
#             print(log_str)
#             print "#################"
            

Loss at step 500 : 3.65587568283
Loss at step 1000 : 2.57884311676
Loss at step 1500 : 2.85987019539
Loss at step 2000 : 3.23281097412
Loss at step 2500 : 3.13484477997
Loss at step 3000 : 2.91896104813
Loss at step 3500 : 2.39592671394
Loss at step 4000 : 2.40252494812
Loss at step 4500 : 3.66019916534
Loss at step 5000 : 3.7126531601
Loss at step 5500 : 2.94248819351
Loss at step 6000 : 3.1342253685
Loss at step 6500 : 3.12881588936
Loss at step 7000 : 2.5771727562
Loss at step 7500 : 2.55633068085
Loss at step 8000 : 2.47573137283
Loss at step 8500 : 2.27930116653
Loss at step 9000 : 2.24365377426
Loss at step 9500 : 2.67922139168
Loss at step 10000 : 2.43516778946
Loss at step 10500 : 3.72865438461
Loss at step 11000 : 2.97700190544
Loss at step 11500 : 2.35210156441
Loss at step 12000 : 2.97778701782
Loss at step 12500 : 2.36637306213
Loss at step 13000 : 2.29116225243
Loss at step 13500 : 2.32501959801
Loss at step 14000 : 2.72837233543
Loss at step 14500 : 3.96937322617
Loss at 

In [36]:
new_df = pd.DataFrame() #creates a new dataframe that's empty
new_df["index"] = range(num_embedding)
import csv
new_df.to_csv(os.path.join(LOG_DIR, 'output2.tsv'), sep='\t', quoting=csv.QUOTE_NONE)



In [37]:
from tensorflow.contrib.tensorboard.plugins import projector
# Create randomly initialized embedding weights which will be trained.

# Format: tensorflow/tensorboard/plugins/projector/projector_config.proto
config = projector.ProjectorConfig()

# You can add multiple embeddings. Here we add only one.
embedding = config.embeddings.add()
embedding.tensor_name = 'product_vec'
# Link this tensor to its metadata file (e.g. labels).
embedding.metadata_path = os.path.join(LOG_DIR, 'output2.tsv')

# Use the same LOG_DIR where you stored your checkpoint.
summary_writer = tf.summary.FileWriter(LOG_DIR)

# The next line writes a projector_config.pbtxt in the LOG_DIR. TensorBoard will
# read this file during startup.
projector.visualize_embeddings(summary_writer, config)

In [38]:
saver = tf.train.Saver()

saver.save(sess, os.path.join(LOG_DIR, "model.ckpt"), 1000)




'/tmp/testing/stock2vec_1/model.ckpt-1000'